<a href="https://www.kaggle.com/code/ryancardwell/seawolfv6-1?scriptVersionId=271361313" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# seawolfv6-1: Integrated ARC solver scaffold (19 modules + 12 knobs + logging)
import json, os, sys, math, random, time
from collections import Counter

SEAWOLF_VERSION = "v6-1"
SEED = 42
random.seed(SEED)

KNOBS = {
    "BEAM_WIDTH": 8,
    "MAX_PERIOD": 8,
    "VETO_THRESH": 0.85,
    "SHAPE_VOTE_TOPK": 3,
    "SEARCH_DEPTH": 2,
    "PALETTE_REROUTE": True,
    "UPSCALE_MIN": (3,3),
    "DYN_PERIOD_HARD_CAP": 8,
    "REPAIR_TO_MOST_COMMON": True,
    "CC_ENABLE": True,
    "SYMMETRY_ENFORCE": True,
    "METRICS_VERBOSITY": 1,
}

def log(msg, lvl=1):
    if KNOBS["METRICS_VERBOSITY"] >= lvl:
        print(f"[SEAWOLF {SEAWOLF_VERSION}] {msg}")


In [2]:
def grid_size(g):
    return (len(g), len(g[0]) if g else 0)

def clone_grid(g):
    return [row[:] for row in g] if g else []

def mode_color(g, default=0):
    if not g or not g[0]: return default
    vals = Counter(v for r in g for v in r)
    return max(vals, key=vals.get)

def palette(g):
    return Counter(v for r in g for v in r)

def valid_grid(g):
    if not isinstance(g, list) or not g or not isinstance(g[0], list):
        return False
    W = len(g[0])
    if any(len(r)!=W for r in g): return False
    if any((v<0 or v>9) for r in g for v in r): return False
    return True


In [3]:
def prevent_degenerate_output(grid, fallback_shape=None, fill_val=0):
    H = len(grid); W = len(grid[0]) if H else 0
    if H >= 2 and W >= 2: 
        return grid
    if not fallback_shape:
        fallback_shape = (max(2,H or 2), max(2,W or 2))
        fallback_shape = (min(fallback_shape[0],5), min(fallback_shape[1],5))
    h, w = fallback_shape
    if H and W:
        return [[grid[r%H][c%W] for c in range(w)] for r in range(h)]
    return [[fill_val for _ in range(w)] for _ in range(h)]


In [4]:
def safe_call(fn, *args, **kwargs):
    try:
        return fn(*args, **kwargs), None
    except Exception as e:
        return None, f"{type(e).__name__}: {e}"

def with_fallback(primary_fn, fb_fn, *args, **kwargs):
    res, err = safe_call(primary_fn, *args, **kwargs)
    if err is None:
        return res, None
    fb_res, fb_err = safe_call(fb_fn, *args, **kwargs)
    return (fb_res if fb_err is None else None), (err if fb_err is None else f"{err} | FB:{fb_err}")


In [5]:
OP_REGISTRY = {}

def op_register(name):
    def deco(fn):
        OP_REGISTRY[name] = fn
        return fn
    return deco

def try_pipeline(inp, ops, ctx):
    g = inp
    for op in ops:
        fn = OP_REGISTRY.get(op)
        if not fn:
            return g, False
        g2, err = safe_call(fn, g, ctx)
        if err: 
            return g, False
        g = g2
    return g, True

def dynamic_candidate_plans(detections):
    plans = []
    if detections.get("sym_any"):
        plans += [["detect_symmetry","apply_symmetry_restore","palette_map","force_shape_op"]]
    plans += [["connected_components","remap_by_example","force_shape_op"]]
    if detections.get("period_hint"):
        plans += [["infer_period","periodic_tile_op","palette_map","force_shape_op"]]
    return plans


In [6]:
def beam_search_over_ops(inp, ctx, candidate_plans, score_fn, width=8):
    scored = []
    for plan in candidate_plans:
        g, ok = try_pipeline(inp, plan, ctx)
        sc = score_fn(g, ctx) if ok else -1e9
        scored.append((sc, plan, g))
    scored.sort(reverse=True, key=lambda t: t[0])
    return scored[:max(1, min(width, len(scored)))]


In [7]:
from math import gcd
from functools import reduce

def lcm(a,b): 
    return a*b//gcd(a,b) if a and b else max(a,b)

def infer_target_shapes_from_train(train_pairs):
    shapes = [grid_size(o) for (_,o) in train_pairs if o and o[0]]
    if not shapes: return []
    from collections import Counter
    ctr = Counter(shapes)
    common = [s for s,_ in ctr.most_common( KNOBS["SHAPE_VOTE_TOPK"] )]
    allH = [h for h,_ in shapes]; allW = [w for _,w in shapes]
    def reduce_lcm(vals):
        if not vals: return 1
        acc = vals[0]
        for v in vals[1:]:
            acc = lcm(acc, v)
        return acc
    H_lcm = reduce_lcm(allH)
    W_lcm = reduce_lcm(allW)
    proposals = list(dict.fromkeys(common + [(H_lcm, W_lcm)]))
    return proposals


In [8]:
def force_shape(grid, H, W, fill=None):
    if H<=0 or W<=0:
        return [[0]]
    if not grid or not grid[0]:
        c = 0 if fill is None else fill
        return [[c for _ in range(W)] for _ in range(H)]
    G = [row[:] for row in grid]
    h, w = len(G), len(G[0])
    if fill is None:
        vals = Counter(v for r in G for v in r)
        fill = max(vals, key=vals.get) if vals else 0
    G = [row[:W] for row in G[:H]]
    while len(G) < H: G.append([fill]*min(w if w>0 else W, W if W>0 else w))
    for i in range(H):
        if len(G[i]) < W:
            G[i].extend([fill]*(W-len(G[i])))
    return G

@op_register("force_shape_op")
def force_shape_op(grid, ctx):
    H,W = ctx.get("target_shape", grid_size(grid))
    return force_shape(grid, H, W), None


In [9]:
def periodic_tile_safe(grid, Ht, Wt, max_period=8):
    H = len(grid); W = len(grid[0]) if H else 0
    if H<=0 or W<=0: return [[0 for _ in range(max(1,Wt))] for _ in range(max(1,Ht))]
    h0 = max(1, min(max_period, H))
    w0 = max(1, min(max_period, W))
    base = [row[:w0] for row in grid[:h0]]
    return [[ base[r%h0][c%w0] for c in range(max(1,Wt))] for r in range(max(1,Ht))]

@op_register("infer_period")
def infer_period(grid, ctx):
    H = len(grid); W = len(grid[0]) if H else 0
    ctx["period"] = (max(1, min(4, H)), max(1, min(4, W)))
    return grid, None

@op_register("periodic_tile_op")
def periodic_tile_op(grid, ctx):
    Ht,Wt = ctx.get("target_shape", grid_size(grid))
    maxp = ctx.get("max_period", KNOBS["MAX_PERIOD"])
    return periodic_tile_safe(grid, Ht, Wt, maxp), None


In [10]:
def choose_target_shape(train_pairs, test_input):
    cands = infer_target_shapes_from_train(train_pairs)
    if not cands:
        return grid_size(test_input)
    def pair_score(shape):
        H,W = shape
        s=0
        for _,out in train_pairs:
            if out:
                s += (H==len(out)) + (W==len(out[0]))
        return s
    cands.sort(key=pair_score, reverse=True)
    return cands[0]


In [11]:
def simple_score_on_train(grid, ctx):
    trs = ctx.get("train", [])
    if not trs or not grid or not grid[0]: return 0.0
    def pal(g): return Counter(v for r in g for v in r)
    pg = pal(grid)
    sc=0
    for _,out in trs:
        if not out: 
            continue
        H,W = len(out), len(out[0])
        sc += (len(grid)==H) + (len(grid[0])==W if grid else 0)
        po = pal(out)
        sc += sum(min(pg[k], po[k]) for k in pg.keys() & po.keys())
    return sc

def run_mini_beam(test_in, ctx, width=None):
    if width is None: width = KNOBS["BEAM_WIDTH"]
    detections = {
        "sym_any": KNOBS["SYMMETRY_ENFORCE"],
        "period_hint": True
    }
    plans = dynamic_candidate_plans(detections)
    topk = beam_search_over_ops(test_in, ctx, plans, simple_score_on_train, width=width)
    return (topk[0][2], topk[0][1], topk[0][0]) if topk else (test_in, [], -1e9)


In [12]:
def veto_or_repair(pred, ctx):
    H,W = ctx.get("target_shape", grid_size(pred))
    ok_shape = grid_size(pred)==(H,W)
    trs = ctx.get("train", [])
    train_pal = None
    if trs and trs[0][1]:
        train_pal = Counter(v for r in trs[0][1] for v in r)

    if not ok_shape:
        pred = force_shape(pred, H, W)

    if KNOBS["PALETTE_REROUTE"] and train_pal:
        pp = palette(pred)
        allowed = set(train_pal.keys())
        if not set(pp.keys()).issubset(allowed):
            to_color = max(train_pal, key=train_pal.get)
            pred = [[(v if v in allowed else to_color) for v in row] for row in pred]
    return pred


In [13]:
def cc_label(grid):
    H = len(grid); W = len(grid[0]) if H else 0
    lab = [[-1]*W for _ in range(H)]
    comps = []
    nid=0
    for r in range(H):
        for c in range(W):
            if lab[r][c] >= 0: continue
            color = grid[r][c]
            q=[(r,c)]; lab[r][c]=nid; pix=[(r,c)]
            while q:
                rr,cc = q.pop()
                for dr,dc in ((1,0),(-1,0),(0,1),(0,-1)):
                    nr,nc=rr+dr, cc+dc
                    if 0<=nr<H and 0<=nc<W and lab[nr][nc]<0 and grid[nr][nc]==color:
                        lab[nr][nc]=nid; q.append((nr,nc)); pix.append((nr,nc))
            comps.append({"id":nid,"color":color,"pixels":pix})
            nid+=1
    return lab, comps

def cc_bboxes(comps):
    bbs=[]
    for c in comps:
        rs=[r for r,_ in c["pixels"]]; cs=[c2 for _,c2 in c["pixels"]]
        bbs.append((min(rs),min(cs),max(rs),max(cs)))
    return bbs

@op_register("connected_components")
def connected_components_op(grid, ctx):
    if KNOBS["CC_ENABLE"]:
        ctx["cc"] = cc_label(grid)
    return grid, None


In [14]:
def is_vsym(g):
    H=len(g); W=len(g[0]) if H else 0
    if H==0 or W==0: return False
    return all(g[r][c]==g[r][W-1-c] for r in range(H) for c in range(W))

def is_hsym(g):
    H=len(g); W=len(g[0]) if H else 0
    if H==0 or W==0: return False
    return all(g[r][c]==g[H-1-r][c] for r in range(H) for c in range(W))

@op_register("detect_symmetry")
def detect_symmetry(grid, ctx):
    ctx["vsym"]=is_vsym(grid); ctx["hsym"]=is_hsym(grid)
    ctx["sym_any"]=ctx["vsym"] or ctx["hsym"]
    return grid, None

@op_register("apply_symmetry_restore")
def apply_symmetry_restore(grid, ctx):
    if not KNOBS["SYMMETRY_ENFORCE"]:
        return grid, None
    H=len(grid); W=len(grid[0]) if H else 0
    g=[row[:] for row in grid]
    if W>0 and ctx.get("vsym"):
        for r in range(H):
            for c in range(W//2):
                g[r][W-1-c]=g[r][c]
    if H>0 and ctx.get("hsym"):
        for r in range(H//2):
            g[H-1-r]=g[r][:]
    return g, None


In [15]:
def learn_palette_map(train_pairs):
    in_cnt=Counter(); out_cnt=Counter()
    for inp,out in train_pairs:
        in_cnt.update(v for r in inp for v in r)
        out_cnt.update(v for r in out for v in r)
    in_sorted=[k for k,_ in in_cnt.most_common()]
    out_sorted=[k for k,_ in out_cnt.most_common()] or in_sorted
    m={}
    for i,k in enumerate(in_sorted):
        m[k]= out_sorted[i%len(out_sorted)] if out_sorted else k
    return m

@op_register("palette_map")
def palette_map_op(grid, ctx):
    m = ctx.get("pal_map")
    if m is None:
        trs = ctx.get("train", [])
        m = learn_palette_map(trs) if trs else {}
        ctx["pal_map"]=m
    return [[m.get(v,v) for v in row] for row in grid], None

@op_register("remap_by_example")
def remap_by_example(grid, ctx):
    return palette_map_op(grid, ctx)


In [16]:
def validate_submission_payload(payload, expected_ids):
    errs=[]
    miss = [i for i in expected_ids if i not in payload]
    if miss: errs.append(f"missing ids: {miss[:8]}{'...' if len(miss)>8 else ''}")
    for tid, grids in payload.items():
        if not isinstance(grids, list): errs.append(f"{tid}: value must be list"); continue
        for g in grids:
            if not valid_grid(g): 
                errs.append(f"{tid}: invalid grid")
                break
    return errs

def write_submission_safe(payload, expected_ids, target_path="submission.json"):
    errs = validate_submission_payload(payload, expected_ids)
    if errs: raise ValueError("Submission invalid: " + " | ".join(errs))
    tmp = target_path + ".tmp"
    with open(tmp,"w",encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, separators=(",",":"))
        f.flush(); os.fsync(f.fileno())
    os.replace(tmp, target_path)
    return target_path


In [17]:
def proxy_score(grid, ctx):
    return simple_score_on_train(grid, ctx)

def grid_search_knobs(task_slice, solver_fn, base_ctx):
    candidates=[]
    for bw in (6,8,10):
        for mp in (4,6,8):
            for veto in (0.75,0.85,0.9):
                ctx = dict(base_ctx, beam_width=bw, max_period=mp, veto_thresh=veto)
                score=0
                for tid,task in task_slice:
                    try:
                        g = solver_fn(task, ctx)[0]
                        score += proxy_score(g, ctx)
                    except Exception:
                        score += 0
                candidates.append((score, ctx))
    candidates.sort(reverse=True, key=lambda t:t[0])
    return candidates[0][1] if candidates else base_ctx


In [18]:
def fix_force_shape_calls(solvers):
    def wrapper_force_shape(grid, *args, **kwargs):
        if len(args)==1 and isinstance(args[0], tuple):
            H,W = args[0]
            return force_shape(grid, H, W, **kwargs)
        elif len(args)==1 and isinstance(args[0], int):
            H = args[0]; W = kwargs.get("W", len(grid[0]) if grid else H)
            return force_shape(grid, H, W, **kwargs)
        elif len(args)==2 and all(isinstance(a,int) for a in args):
            return force_shape(grid, args[0], args[1], **kwargs)
        else:
            return force_shape(grid, *args, **kwargs)
    solvers["force_shape"] = wrapper_force_shape
    return solvers

def patch_periodic_tile_refs(solvers):
    solvers["periodic_tile"] = periodic_tile_safe
    return solvers

def attach_shape_inference(ctx):
    trs = ctx.get("train", [])
    if trs:
        ctx["target_shape"] = choose_target_shape(trs, ctx.get("test_in"))
    else:
        tin = ctx.get("test_in", [])
        ctx["target_shape"] = grid_size(tin)
    return ctx


In [19]:
def solve_with_beam_and_veto(task, override_ctx=None):
    train_pairs = task.get("train", [])
    test_in = task["test"][0]
    ctx = {"train": train_pairs, "test_in": test_in}
    ctx = attach_shape_inference(ctx)
    ctx["max_period"]= KNOBS["MAX_PERIOD"]
    ctx["pal_map"]=learn_palette_map(train_pairs) if train_pairs else {}
    if override_ctx:
        ctx.update(override_ctx)
    pred, plan, score = run_mini_beam(test_in, ctx, width=KNOBS["BEAM_WIDTH"])
    pred = veto_or_repair(pred, ctx)
    pred = prevent_degenerate_output(pred, ctx["target_shape"])
    return [pred]

def finalize_and_save_submission(TEST_TASKS, solver_fn, expected_ids, path="submission.json"):
    payload={}
    t0=time.time()
    for i,(tid, task) in enumerate(TEST_TASKS.items()):
        try:
            outs = solver_fn(task)
            if not isinstance(outs, list): 
                outs=[outs]
            payload[tid] = outs
            if (i+1)%25==0:
                log(f"[PROGRESS] {i+1}/{len(TEST_TASKS)} tasks solved...", lvl=1)
        except Exception as e:
            log(f"[WARN] {tid}: solver exception -> {e}", lvl=1)
            payload[tid] = [[[0]]]
    path = write_submission_safe(payload, expected_ids, path)
    log(f"[SUBMISSION] wrote {path} in {time.time()-t0:.2f}s", lvl=1)
    return path

# Harness: replace TEST_TASKS with real loader in Kaggle
TEST_TASKS = {}
solvers = {}
solvers = fix_force_shape_calls(solvers)
solvers = patch_periodic_tile_refs(solvers)

def build_expected_ids(TEST_TASKS):
    return list(TEST_TASKS.keys())

log("Notebook skeleton ready. Plug in your ARC JSON loaders, then call finalize_and_save_submission(...).", 1)


[SEAWOLF v6-1] Notebook skeleton ready. Plug in your ARC JSON loaders, then call finalize_and_save_submission(...).
